#### Script for downloading and processing the HA sequences from the four serotypes of interest (H1, H3N2, H5 and H7)

<br>
<br>


In [1]:
import pandas as pd
from Bio import SeqIO, Seq, Entrez
import numpy as np
import datetime
import ete3
import re
import math
import statistics
import random

Entrez.email = ''

In [2]:
def filt_fas(recdic, acclist, outname):
    fasta = ''
    for rec in list(recdic):
        shrec = rec.split('|')[0]
        if shrec in acclist:
            fasta = fasta + '>%s\n%s\n'%(shrec, str(recdic[rec].seq))
    print(fasta.count('>'))
    with open(outname, 'w') as out:
        out.write(fasta[:-1])

In [3]:
def gapaway_plus(file, ungapprop, ungaptype):  # ungaptype = seqnum|seqprop

    recdic = SeqIO.to_dict(SeqIO.parse(file, 'fasta'))
    al_len = len(recdic[list(recdic)[0]].seq)
    gaplocs = []
    for i in range(al_len):
        gcheck = []
        for rec in list(recdic):
            gcheck.append(str(recdic[rec].seq)[i])          
        if ungaptype == 'seqprop':

            if (gcheck.count('-')/len(gcheck))*100 > ungapprop:

                gaplocs.append(i)
                
        elif ungaptype == 'seqnum':
            
            if  len(gcheck) - gcheck.count('-') <= ungapprop:

                gaplocs.append(i)            

        if i%100 == 0:
            print(i+1)

    fasta = ''

    for rec in recdic:

        theseq = str(recdic[rec].seq)

        for i in range(len(gaplocs)):

            theseq = theseq[0:gaplocs[i]-i] + theseq[gaplocs[i]-i+1:]

        fasta = fasta + '>%s\n%s\n'%(recdic[rec].description, theseq)

    with open(file.replace('.fas', '_%ipungap.fas'%ungapprop), 'w+') as out:
        out.write(fasta[:-1])

In [4]:
def date_to_decimal_year(date_str):
    
    # Parse the input date string into a datetime object
    dt = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    
    # Calculate the decimal year
    decimal_year = dt.year + (dt.month - 1) / 12 + (dt.day - 1) / 365
    
    return decimal_year

In [5]:
def ntal2df(gsdseq, fileid, thepath):
    
    ghadic = {rec.id:rec for rec in SeqIO.parse(thepath + '/' + gsdseq, 'fasta')}
    
    hall_col = ['acc','name', 'strain', 'seg', 'date', 'lineage', 'clade', 'type', 'cds', 'aa']
    hall_d = {x:[] for x in hall_col}
    
    for rec in list(ghadic):
        #make sure header has info for all fields
        if len(rec.split('|')) == len(hall_col)-2:
    
            #iterate through header info
            for i in range(len(hall_col)-2):
                hall_d[hall_col[i]].append(rec.split('|')[i])
            thecds = str(ghadic[rec].seq).replace('-', '')
    
            #add CDS and translation to dict
            hall_d['cds'].append(thecds)
            hall_d['aa'].append(str(Seq.Seq(thecds).translate()))
        
        
    hall_df = pd.DataFrame(hall_d)
    
    #fix type
    hall_df['type'] = [x.replace('A_/_', '') for x in list(hall_df.type)]
    
    #filter out aa seqs with stop codons and Ns
    hall_df = hall_df[(~hall_df.aa.str.contains("\*")) & (~hall_df.aa.str.contains('X'))]
    
    #filter out aa seqs that don't start with M
    hall_df = hall_df[hall_df.aa.str[0] == 'M']
    
    #get numerical dates
    hall_df['numdate'] = [date_to_decimal_year(x) for x in list(hall_df.date)]
    
    #sort and drop duplicates on aa level keeping the earliest one
    hall_df.sort_values(by='numdate', inplace=True)
    hall_df.drop_duplicates(subset='aa', inplace=True)

    #filter by length of aa sequence to only get complete ones
    hall_df = hall_df[hall_df.aa.str.len() >= 550]    
    

    hall_df[['acc', 'strain', 'date', 'clade', 'type', 'numdate']].to_csv('%s/%s_filt_meta.csv'%(thepath, fileid), index=False)
    hall_df[['acc', 'strain', 'date', 'clade', 'type', 'numdate']].to_csv('%s/%s_filt_meta.tsv'%(thepath, fileid), index=False, sep='\t')
    
    
    with open('%s/%s_filt_cds.fas'%(thepath, fileid), 'w') as out:
        out.write('\n'.join(['>%s\n%s'%(a,s) for a,s in zip(list(hall_df.acc), list(hall_df.cds))]))
    with open('%s/%s_filt_aa.fas'%(thepath, fileid), 'w') as out:
        out.write('\n'.join(['>%s\n%s'%(a,s) for a,s in zip(list(hall_df.acc), list(hall_df.aa))]))

    #for timetree
    hall_df.rename(columns={'name':'strainname','acc':'name', 'date':'datefull', 'numdate':'date'}, inplace=True)
    hall_df[['name', 'date']].to_csv('%s/%s_filt_dates.tsv'%(thepath, fileid), index=False, sep='\t')    
   
    return hall_df

In [6]:
def treetime_sum(ttpath, datatype):
    
    nodedetsl = []

    asrdic = SeqIO.to_dict(SeqIO.parse(ttpath + '/ancestral_sequences.fasta', 'fasta'))

    with open(ttpath + '/timetree.nexus', 'r') as nxstree:
        nwkf = [l for l in nxstree.readlines()][6].replace(' Tree tree1=', '')
        for x in re.findall(r"NODE.*?\]", nwkf):
    #         print(x)
            if '&mutations=\"' in x:
                nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), x.split('&mutations=\"')[1].split('\"')[0], float(x.split('date=')[1][:-1]) , str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )
            else:
                nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), "", float(x.split('date=')[1][:-1]), str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )

        if datatype == 'gisaid':
            for x in re.findall(r"EPI_ISL_[A-Za-z0-9_]*:[^\]]*\]", nwkf):
                if '&mutations=\"' in x:
                    nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), x.split('&mutations=\"')[1].split('\"')[0], float(x.split('date=')[1][:-1]) , str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )
                else:
                    nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), "", float(x.split('date=')[1][:-1]), str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )
        elif datatype == 'ncbi':
            for x in re.findall(r"[A-Za-z0-9]{8}:[^\]]*\]", nwkf):
                if '&mutations=\"' in x:
                    nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), x.split('&mutations=\"')[1].split('\"')[0], float(x.split('date=')[1][:-1]) , str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )
                else:
                    nodedetsl.append( [x.split(':')[0], float(x.split(':')[1].split('[')[0]), "", float(x.split('date=')[1][:-1]), str(asrdic[x.split(':')[0]].seq).replace('-', '') ] )

                    
        else:
            print('datatype should be ncbi or gisaid')


    nodedetsdf = pd.DataFrame(nodedetsl)
    nodedetsdf.columns = ['node', 'branchlen', 'subs', 'date', 'seq']
    nodedetsdf['seqlen'] = nodedetsdf.seq.str.len()
    nodedetsdf.to_csv(ttpath + '/%s_nodedetails.csv'%ttpath.split('/')[-1], index=False)

    return nodedetsdf


In [7]:
def rm_outliers(treetimedir, fileid):
    outl = list(pd.read_csv('%s/outliers.tsv'%treetimedir , sep='\t')["Unnamed: 0"])
    print(len(outl))
    
    nwkf = ''
    with open('%s/timetree.nexus'%treetimedir, 'r') as nxstree:
        nwkf = [l for l in nxstree.readlines()][6].replace(' Tree tree1=', '')
        nwkf = re.sub(r'\[.*?\]', '', nwkf)
    
    thet = ete3.Tree(nwkf, format=1)
    remain = list(set(thet.get_leaf_names()) - set(outl))
    
    print(len(thet))
    thet.prune(remain, 'preserve_branch_length')
    print(len(thet))
    thet.write(outfile='%s/%s_tf.nwk'%(treetimedir, fileid), format=1)
    


<br>
<br>
<br>
<br>
<br>

### GISAID - H5/H7

Function reads the preliminary alignment of gisaid retrieved nt sequences `al_gisaid_h5_270624_nt_trim.fasta` and makes filtered:

- metadata tsv
- dates tsv
- cds fasta
- aa fasta

In [14]:
h5df = ntal2df('al_gisaid_h5_270624_nt_trim.fasta', 'gisaid_h5_270624', 'data')
h5df

C:\Users\spyro\anaconda3\Lib\site-packages\Bio\Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,name,strainname,strain,seg,datefull,lineage,clade,type,cds,aa,date
1437,EPI_ISL_69953,A/shearwater/Australia/751/1975,A/shearwater/Australia/751/1975,HA,1975-01-01,,EA_nonGsGD,H5N3,atggagagagtagtgcttcttcttgcaatgatcagtcttgtcaaaa...,MERVVLLLAMISLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,1975.000000
2438,EPI_ISL_22545,A/mallard/WI/42/1975,A/mallard/WI/42/1975,HA,1975-01-01,,Am_nonGsGD,H5N2,atggaaagaatagtgattgcccttgcaataatcagcgttgtcaaag...,MERIVIALAIISVVKGDQICIGYHANNSTKQVDTIMEKNVTVTHAQ...,1975.000000
3585,EPI_ISL_17768705,A/mallard/Alabama/847/1975_HA,A/mallard/Alabama/847/1975,HA,1975-01-01,,Am_nonGsGD,H5N2,atggaaagaatagtgattgcccttgcaataatcagcattgtcaaag...,MERIVIALAIISIVKGDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,1975.000000
3584,EPI_ISL_17768706,A/mallard/Wisconsin/42/1975_HA,A/mallard/Wisconsin/42/1975,HA,1975-01-01,,Am_nonGsGD,H5N2,atggaaagaatagtgattgcccttgcaataatcagcgttgtcaaag...,MERIVIALAIISVVKGDQICIGYHANNSTKQVDTIMEKNVTVTHAQ...,1975.000000
5716,EPI_ISL_12517,A/blue_goose/WI/711/1975,A/blue_goose/WI/711/1975,HA,1975-01-01,,Am_nonGsGD,H5N2,atggaaagaatagtgattgcccttgcaataatcagcattgtcaaag...,MERIVIALAIISIVKGDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,1975.000000
...,...,...,...,...,...,...,...,...,...,...,...
23495,EPI_ISL_19132453,A/Fujian/1/2024(H5N6),A/Fujian/1/2024(H5N6),HA,2024-04-23,,2.3.4.4h,H5N6,atggagaaaatagtacttcttctttcagtggtgaaccttgtcaaaa...,MEKIVLLLSVVNLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,2024.310274
17165,EPI_ISL_19186450,A/State_of_Mexico/INER-INF645/2024,A/State_of_Mexico/INER-INF645/2024,HA,2024-04-24,,EA_nonGsGD,H5N2,atggaaagaatagtgattgctttcgcaataatcagcattgtcgcag...,MERIVIAFAIISIVAGDKICIGYHANNSTKQVDTIMEKNVTVTHAQ...,2024.313014
22955,EPI_ISL_19214219,A/Gallus_gallus/Libres/CPA-05168-24/2024_HA,A/Gallus_gallus/Puebla/CPA-05168-24/2024,HA,2024-04-29,,Am_nonGsGD,H5N2,atggaaagaatagtgattgctttcgcaataatcagcattgtcgcag...,MERIVIAFAIISIVAGDKICIGYHANNSTKQVDTIMEKNVTVTHAQ...,2024.326712
21168,EPI_ISL_19210416,A/sandwich_tern/Spain/1662-3_24VIR4860-18/2024_HA,A/sandwich_tern/Spain/1662-3_24VIR4860-18/2024,HA,2024-05-10,,2.3.4.4b,H5N1,atggagaacatagtacttcttcttgcaatagttagttttgttaaaa...,MENIVLLLAIVSFVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,2024.357991


In [161]:
h7df = ntal2df('al_gisaid_h7_270624_nt_trim.fasta', 'gisaid_h7_270624', 'data')
h7df

C:\Users\spyro\anaconda3\Lib\site-packages\Bio\Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,name,strainname,strain,seg,datefull,lineage,clade,type,cds,aa,date
2839,EPI_ISL_69853,A/chicken/Brescia/1902,A/chicken/Brescia/1902,HA,1902-01-01,,,H7N7,atgaacactcaaatcctggtatttgcccttgtggcagtcatccaca...,MNTQILVFALVAVIHTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,1902.000000
1000,EPI_ISL_132870,A/FPV/Dutch/1927,A/FPV/Dutch/1927,HA,1927-01-01,,,H7N7,atgaacactcaaatcctggtattcgcccttgtggcagtcatccaca...,MNTQILVFALVAVIHTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,1927.000000
2768,EPI_ISL_69760,A/FPV/Dutch/27,A/FPV/Dutch/27,HA,1927-01-12,,,H7N7,atgaacactcaaatcctggtattcgcccttgtggcagtcattcaca...,MNTQILVFALVAVIHTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,1927.030137
364,EPI_ISL_66098,A/fowl/Weybridge,A/fowl/Weybridge,HA,1933-12-31,,,H7N7,atgaacactcaaatcctggtattcgcccttgtggcagtcatccaca...,MNTQILVFALVAVIHTNADKICLGHHVSSNGTKVNTLTERGVEVVN...,1933.998858
357,EPI_ISL_66097,A/FPV/Rostock/1934,A/FPV/Rostock/1934,HA,1933-12-31,,,H7N1,atgaacactcaaatcctggttttcgcccttgtggcagtcattccca...,MNTQILVFALVAVIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,1933.998858
...,...,...,...,...,...,...,...,...,...,...,...
1420,EPI_ISL_18910108,A/environment/Kagoshima/KU-B17/2023,A/environment/Kagoshima/KU-B17/2023,HA,2024-01-15,,,H7N7,atgaacactcaaatcctggtactcgctctagtggcgataattccaa...,MNTQILVLALVAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,2024.038356
1818,EPI_ISL_19180882,A/duck/Bangladesh/WF-520/2024_HA,A/duck/Bangladesh/WF-520/2024,HA,2024-01-22,,,H7N7,atgaacactcaaatcctggtattcgctctgattgcgatcattccaa...,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,2024.057534
1821,EPI_ISL_19180883,A/duck/Bangladesh/WF-529/2024_HA,A/duck/Bangladesh/WF-529/2024,HA,2024-01-22,,,H7N7,atgaacactcaaatcctggtattcgctctgattgcgatcattccaa...,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,2024.057534
1809,EPI_ISL_19180884,A/duck/Bangladesh/WF-562/2024_HA,A/duck/Bangladesh/WF-562/2024,HA,2024-01-23,,,H7,atgaacactcaaatcctggtattcgctctgattgcgatcattccaa...,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,2024.060274


In [168]:
#need to manually edit the date for the Brescia strain, should be 1935 not 1902 https://www.jstor.org/stable/3298866
h7dates = pd.read_csv('data/gisaid_h7_270624_filt_dates.tsv', sep='\t')
h7dates.loc[0,'date'] = 1935.0
h7dates.sort_values(by='date', inplace=True)
h7dates.to_csv('data/gisaid_h7_270624_filt_dates.tsv', sep='\t', index=False)
h7dates

,name,date
1,EPI_ISL_132870,1927.000000
2,EPI_ISL_69760,1927.030137
3,EPI_ISL_66098,1933.998858
4,EPI_ISL_66097,1933.998858
5,EPI_ISL_5878,1933.998858
...,...,...
2224,EPI_ISL_18910108,2024.038356
2225,EPI_ISL_19180882,2024.057534
2226,EPI_ISL_19180883,2024.057534
2227,EPI_ISL_19180884,2024.060274


<br>
<br>

### NCBI - H1/H3

Uses the original `ncbiflu_HA_all_110424_noX.csv` data used for model training to download cds sequences and retrieve the same files as above (sequence fasta & metadata)

In [9]:
allncbidf = pd.read_csv('train_data/ncbiflu_HA_all_110424_noX.csv')
allncbidf

,name,strain,date,host,serotype,country,tax,seq,lens
0,QBM69670,A/China/38888/2012,2012-08-29,Human,H3N2,China,Influenza A virus (A/China/38888/2012),MKTIIALSYILCLVVAQKLPGNDNSTATLCLGHHAVPNGTIVKTIT...,566
1,AIY59238,A/duck/Shandong/TA01/2012,2012-12-27,Avian,H9N2,China,Influenza A virus (A/duck/Shandong/TA01/2012(H...,METVSLITILLVATVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,560
2,ADE20868,A/Wisconsin/629-S1414/2009,2009-10-26,Human,H1N1,USA,Influenza A virus (A/Wisconsin/629-S1414/2009(...,MKAILVVLLYTFATANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,566
3,ADK78207,A/rosy-billed pochard/Argentina/CIP051-925/2008,2008-02-,Avian,H6N2,Argentina,Influenza A virus (A/rosy-billed pochard/Argen...,MIAIIVIAILATAGRSDKICIGYHANNSTTQVDTILEKNVTVTHSV...,566
4,ADW80272,A/mallard/Ohio/662/2002,2002-08-19,Avian,"mixed,H6",USA,Influenza A virus (A/mallard/Ohio/662/2002(mix...,MIAIIILAILVSTSKSDRICIGYHANNSTTQVDTILEKNVTVTHSV...,566
...,...,...,...,...,...,...,...,...,...
33652,WJD58476,A/swine/Kansas/A02861449/2023,2023-05-03,Swine,H3N2,USA,Influenza A virus (A/swine/Kansas/A02861449/2023),MKTIIAFSCILCQISSQNLPGSDNSMATLCLGHHAVPNGTLVKTIT...,566
33653,WJN08972,A/California/42/2023,2023-03-06,Human,H3N2,USA,Influenza A virus (A/California/42/2023),MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566
33654,WJN09188,A/Colorado/39/2023,2023-03-21,Human,H3N2,USA,Influenza A virus (A/Colorado/39/2023),MKAIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566
33655,WJN09500,A/Maryland/17/2023,2023-02-06,Human,H3N2,USA,Influenza A virus (A/Maryland/17/2023),MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566


In [162]:
h3df = allncbidf[(allncbidf.host == 'Human') & (allncbidf.serotype == 'H3N2')]
h3df = h3df[h3df.date != 'unknown--']

newdatl = []
for x in list(h3df['date']):
    if len(x) == 10:
        newdatl.append(date_to_decimal_year(x))
    elif len(x) == 8:
        newdatl.append(date_to_decimal_year(x + '01'))
    elif len(x) == 6:
        newdatl.append(date_to_decimal_year(x[:-2] + '-01-01'))

h3df['numdate'] = newdatl

h3df.sort_values(by='numdate', inplace=True)


downhandle = Entrez.efetch(db="protein", id=list(h3df.name)[:9999], rettype="fasta_cds_na", retmode="text")
with open('data/ncbi_h3n2_110424_filt_cds_1.fas', 'w') as out:
    out.write(downhandle.read())

downhandle = Entrez.efetch(db="protein", id=list(h3df.name)[9999:], rettype="fasta_cds_na", retmode="text")
with open('data/ncbi_h3n2_110424_filt_cds_2.fas', 'w') as out:
    out.write(downhandle.read())

fnacc = []
aafas = ''
cdsfas = ''
for i in range(1,3):
    for rec in SeqIO.parse('data/ncbi_h3n2_110424_filt_cds_%i.fas'%i, 'fasta'):
        thecds = str(rec.seq)[:-3]
        theaa = str(rec.seq.translate())[:-1]
        if ('*' not in theaa) and (len(thecds)%3==0) and ('X' not in theaa) and (theaa[0] == 'M'):
            fnacc.append(str(rec.id).split('_cds_')[1].split('.')[0])
            cdsfas = cdsfas + '>%s\n%s\n'%(str(rec.id).split('_cds_')[1].split('.')[0], thecds)
            aafas = aafas + '>%s\n%s\n'%(str(rec.id).split('_cds_')[1].split('.')[0], theaa)

with open('data/ncbi_h3n2_110424_filt_cds.fas', 'w') as out:
    out.write(cdsfas[:-1])
with open('data/ncbi_h3n2_110424_filt_aa.fas', 'w') as out:
    out.write(aafas[:-1])

h3df = h3df[h3df.name.isin(fnacc)]
        
h3df.rename(columns={'name':'acc','serotype':'type'}, inplace=True)

h3df[['acc', 'strain', 'date', 'type', 'numdate', 'host', 'country']].to_csv('data/ncbi_h3n2_110424_filt_meta.csv', index=False)
h3df[['acc', 'strain', 'date', 'type', 'numdate', 'host', 'country']].to_csv('data/ncbi_h3n2_110424_filt_meta.tsv', index=False, sep='\t')

h3df.rename(columns={'name':'strainname','acc':'name', 'date':'datefull', 'numdate':'date'}, inplace=True)
h3df[['name', 'date']].to_csv('data/ncbi_h3n2_110424_filt_dates.tsv', index=False, sep='\t')  

h3df

,name,strain,datefull,host,type,country,tax,seq,lens,date
23044,ABQ97200,A/Hong Kong/68,1968--,Human,H3N2,Hong Kong,Influenza A virus (A/Hong Kong/68(H3N2)),MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566,1968.000000
15334,ABF83447,A/Northern Territory/60/1968,1968--,Human,H3N2,Australia,Influenza A virus (A/Northern Territory/60/196...,MKTIIALSYIFCLALGQDLPGNDNNTATLCLGHHAVPNGTLVKTIT...,566,1968.000000
15451,BAF48361,A/Aichi/2/1968,1968--,Human,H3N2,Japan,Influenza A virus (A/Aichi/2/1968(H3N2)),MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566,1968.000000
974,ABO52357,A/Albany/17/1968,1968--,Human,H3N2,USA,Influenza A virus (A/Albany/17/1968(H3N2)),MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566,1968.000000
15037,CAA24269,A/Aichi/2/1968,1968--,Human,H3N2,Japan,Influenza A virus (A/Aichi/2/1968(H3N2)),MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566,1968.000000
...,...,...,...,...,...,...,...,...,...,...
22533,WJN09452,A/Hawaii/19/2023,2023-03-23,Human,H3N2,USA,Influenza A virus (A/Hawaii/19/2023),MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566,2023.226941
22531,WJJ08286,A/Rheinland-Pfalz/USAFSAM-14189/2023,2023-03-24,Human,H3N2,Germany,Influenza A virus (A/Rheinland-Pfalz/USAFSAM-1...,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566,2023.229680
7495,WJN10242,A/Texas/11/2023,2023-03-25,Human,H3N2,USA,Influenza A virus (A/Texas/11/2023),MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566,2023.232420
29999,WJN10456,A/Wisconsin/36/2023,2023-03-30,Human,H3N2,USA,Influenza A virus (A/Wisconsin/36/2023),MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566,2023.246119


In [163]:
h1df = allncbidf[(allncbidf.serotype.str[:3] == 'H1N')] 
h1df =  h1df[(h1df.host == 'Human') | (h1df.host == 'Swine')]
h1df = h1df[~h1df.date.str.contains('nknown--')]
newdatl = []
for x in list(h1df['date']):
    if len(x) == 10:
        newdatl.append(date_to_decimal_year(x))
    elif len(x) == 8:
        newdatl.append(date_to_decimal_year(x + '01'))
    elif len(x) == 6:
        newdatl.append(date_to_decimal_year(x[:-2] + '-01-01'))
    else:
        print(x)

h1df['numdate'] = newdatl

h1df.sort_values(by='numdate', inplace=True)


downhandle = Entrez.efetch(db="protein", id=list(h1df.name), rettype="fasta_cds_na", retmode="text")
with open('data/ncbi_h1_110424_filt_cds_0.fas', 'w') as out:
    out.write(downhandle.read())


fnacc = []
aafas = ''
cdsfas = ''
for rec in SeqIO.parse('data/ncbi_h1_110424_filt_cds_0.fas', 'fasta'):
    thecds = str(rec.seq)[:-3]
    theaa = str(rec.seq.translate())[:-1]
    if ('*' not in theaa) and (len(thecds)%3==0) and ('X' not in theaa) and (theaa[0] == 'M'):
        fnacc.append(str(rec.id).split('_cds_')[1].split('.')[0])
        cdsfas = cdsfas + '>%s\n%s\n'%(str(rec.id).split('_cds_')[1].split('.')[0], thecds)
        aafas = aafas + '>%s\n%s\n'%(str(rec.id).split('_cds_')[1].split('.')[0], theaa)

with open('data/ncbi_h1_110424_filt_cds.fas', 'w') as out:
    out.write(cdsfas[:-1])
with open('data/ncbi_h1_110424_filt_aa.fas', 'w') as out:
    out.write(aafas[:-1])

h1df = h1df[h1df.name.isin(fnacc)]

h1df.rename(columns={'name':'acc','serotype':'type'}, inplace=True)

h1df[['acc', 'strain', 'date', 'type', 'numdate', 'host', 'country']].to_csv('data/ncbi_h1_110424_filt_meta.csv', index=False)
h1df[['acc', 'strain', 'date', 'type', 'numdate', 'host', 'country']].to_csv('data/ncbi_h1_110424_filt_meta.tsv', index=False, sep='\t')

h1df.rename(columns={'name':'strainname','acc':'name', 'date':'datefull', 'numdate':'date'}, inplace=True)
h1df[['name', 'date']].to_csv('data/ncbi_h1_110424_filt_dates.tsv', index=False, sep='\t')  

h1df

C:\Users\spyro\anaconda3\Lib\site-packages\Bio\Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,name,strain,datefull,host,type,country,tax,seq,lens,date
15065,AAD17229,A/South Carolina/1/1918,1918--,Human,H1N1,USA,Influenza A virus (A/South Carolina/1/1918(H1N1)),MEARLLVLLCAFAATNADTICIGYHANNSTDTVDTVLEKNVTVTHS...,566,1918.000000
1994,ABV25634,A/swine/Iowa/15/1930,1930--,Swine,H1N1,USA,Influenza A virus (A/swine/Iowa/15/1930(H1N1)),MKAILLVLLCAFAATNADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,566,1930.000000
1055,ACV52101,A/swine/USA/1976/1931,1931--,Swine,H1N1,USA,Influenza A virus (A/swine/USA/1976/1931(H1N1)),MKARLLVLLCAFAATNADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,566,1931.000000
30501,ACV52112,A/swine/USA/1976-MA/1931,1931--,Swine,H1N1,USA,Influenza A virus (A/swine/USA/1976-MA/1931(H1...,MKARLLVLLCAFAATNADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,566,1931.000000
3744,AAA92280,A/WS/1933,1933--,Human,H1N1,Australia,Influenza A virus (A/WS/1933(H1N1)),MKAKLLVLLCALAATDADTICIGYHANNSTDTVDTLLEKNVTVTHS...,566,1933.000000
...,...,...,...,...,...,...,...,...,...,...
29323,QOY39392,A/swine/Italy/161449/2017,2017-12-07,Swine,H1N1,Italy,Influenza A virus (A/swine/Italy/161449/2017),MKTVIALSYFFCLVFGQDFPGKGNNTATLCLGHHAVPNGTLVKTIT...,566,2017.933105
21858,QOY39462,A/swine/Italy/134758/2017,2017-12-30,Swine,H1N1,Italy,Influenza A virus (A/swine/Italy/134758/2017),MKTVIALSYVFCLVFGQDFPGKGNNTATLCLGHHAVPNGTLVKTIT...,566,2017.996119
3559,QJD57231,A/swine/Thailand/CU22337/2018,2018-10-,Swine,H1N1,Thailand,Influenza A virus (A/swine/Thailand/CU22337/2018),MKTIIALSYILCLVFTQKIPGNDNSKATLCLGHHAVSNGTLVKTIT...,566,2018.750000
14416,QRI43229,A/swine/Minnesota/A02245867/2021,2021-01-07,Swine,H1N2,USA,Influenza A virus (A/swine/Minnesota/A02245867...,MKTIIAFSCILCLIIAQKLPGSDNSMATLCLGHHAVPNGTLVKTIT...,566,2021.016438


<br>

### Tree inference

Align:

`mafft --thread 20 --threadtb 20 --threadit 20 --reorder --op 7 gisaid_h7_270624_filt_aa.fas > gisaid_h7_270624_filt_aa.aln.fas`

`mafft --thread 20 --threadtb 20 --threadit 20 --reorder --op 10 gisaid_h5_270624_filt_aa.fas > gisaid_h5_270624_filt_aa.aln.fas`

`mafft --thread 20 --threadtb 20 --threadit 20 --reorder ncbi_h1_110424_filt_aa.fas > ncbi_h1_110424_filt_aa.aln.fas`

`mafft --thread 20 --threadtb 20 --threadit 20 --reorder ncbi_h3n2_110424_filt_aa.fas > ncbi_h3n2_110424_filt_aa.aln.fas`

pal2nal: 

`pal2nal gisaid_h7_270624_filt_aa.aln.fas gisaid_h7_270624_filt_cds.fas -output fasta > gisaid_h7_270624_filt_cds.aln.fas`

Tree:

`iqtree2 -nt 24 -s gisaid_h7_270624_filt_cds.aln.fas -m GTR+F+R4`

ASR:

`treetime --tree gisaid_h7_270624_filt_cds.aln.fas.treefile --aln gisaid_h7_270624_filt_aa.aln.fas --dates gisaid_h7_270624_filt_dates.tsv --aa --outdir gisaid_h7_270624_filt`

<br>

<br>

### Tree parsing

Use the time-calibrated tree inference and protein sequence ancestral reconstruction to get:

- csv with ancestral sequence, and metadata for each tree node
- timetree without the root2tip regression outliers


In [32]:
h7treetimedf = treetime_sum('data/gisaid_h7_270624_filt', 'gisaid')
h7treetimedf.seqlen.value_counts()

seqlen
560    3001
564     318
552     203
562     155
568      63
570      51
563      43
567       7
561       5
569       4
559       1
566       1
565       1
Name: count, dtype: int64

In [33]:
rm_outliers('data/gisaid_h7_270624_filt', 'gisaid_h7_270624_filt')

55
2229
2174


In [7]:
h5treetimedf = treetime_sum('data/gisaid_h5_270624_filt', 'gisaid')
h5treetimedf

,node,branchlen,subs,date,seq,seqlen
0,NODE_0000022,0.29111,"I11T,K171Q",2004.11,MERIVIALAITSIVKADQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,564
1,NODE_0000026,0.24961,T541I,2010.76,MERIVIALAIISIVKADQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,564
2,NODE_0000028,0.20717,,2010.71,MERIVIALAIISIVKADQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,564
3,NODE_0000025,1.18875,,2010.51,MERIVIALAIISIVKADQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,564
4,NODE_0000024,3.33987,"V300I,R481K,N504D",2009.32,MERIVIALAIISIVKADQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,564
...,...,...,...,...,...,...
15064,EPI_ISL_19159885,0.00000,,2023.79,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,567
15065,EPI_ISL_18691856,0.21169,"T143A,I180V",2024.00,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,567
15066,EPI_ISL_18697726,0.15022,V5A,2023.81,MENIALLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,567
15067,EPI_ISL_19064389,0.00000,,2023.82,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,567


In [22]:
rm_outliers('data/gisaid_h5_270624_filt', 'gisaid_h5_270624_filt')

547
8977
8430


In [60]:
h1treetimedf = treetime_sum('data/ncbi_h1_110424_filt', 'ncbi')
h1treetimedf

,node,branchlen,subs,date,seq,seqlen
0,NODE_0002086,3.09304,"R77Q,N159D,S188L,G189N,T229I,I350V,E520G",2004.48,MKTIIALSYILCLVFAQKLPGNDNSMATLCLGHHAVPNGTLVKTIT...,567
1,NODE_0002085,15.11965,"T26M,F62S,E126D,S165Y,E227D,E531V",2001.39,MKTIIALSYILCLVFAQKLPGNDNSMATLCLGHHAVPNGTLVKTIT...,567
2,NODE_0002090,0.51807,I279V,2016.77,MKTVIALSYVFCLVFGQDFPGKGNNTATLCLGHHAVPNGTLVKTIT...,567
3,NODE_0002091,0.72017,I402T,2016.97,MKTVIALSYVFCLVFGQDFPGKGNNTATLCLGHHAVPNGTLVKTIT...,567
4,NODE_0002089,0.10261,,2016.25,MKTVIALSYVFCLVFGQDFPGKGNNTATLCLGHHAVPNGTLVKTIT...,567
...,...,...,...,...,...,...
13922,ARH17613,0.60056,P322L,2017.11,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,568
13923,APO20445,0.28914,P311S,2016.80,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,568
13924,AQS98486,0.55284,"I5T,E95D",2017.06,MKATLVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,568
13925,APO20456,0.00000,,2016.85,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,568


In [61]:
rm_outliers('data/ncbi_h1_110424_filt', 'ncbi_h1_110424_filt')

413
8191
7778


In [62]:
h3treetimedf = treetime_sum('data/ncbi_h3n2_110424_filt', 'ncbi')
h3treetimedf

,node,branchlen,subs,date,seq,seqlen
0,NODE_0000004,0.11299,L348I,1967.66,MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566
1,NODE_0000006,0.10525,G235E,1967.65,MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566
2,NODE_0000011,0.18555,D478E,1967.95,MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566
3,NODE_0000010,0.21554,V199I,1967.76,MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTIT...,566
4,NODE_0000026,0.27832,,1970.90,MKTIIALSYIFYLALGQDLPGNDNSKATLCLGHHAVPNGTLVKTIT...,566
...,...,...,...,...,...,...
16952,WAJ06421,0.14689,R278Q,2022.26,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566
16953,USW63032,0.15237,N39D,2022.27,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPDGTIVKTIT...,566
16954,UUB81155,0.16607,L445I,2022.28,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566
16955,UUV81804,0.16607,Y528H,2022.28,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,566


In [63]:
rm_outliers('data/ncbi_h3n2_110424_filt', 'ncbi_h3n2_110424_filt')

55
10151
10096
